In [1]:
import scanpy as sc
import squidpy as sq

import numpy as np
import pandas as pd

In [2]:
adata = sc.read_h5ad('datasets/slidetag_LN/slidetag_LN.h5ad')

In [3]:
from simvi.model import SimVI
SimVI.setup_anndata(adata)
edge_index = SimVI.extract_edge_index(adata,n_neighbors=10)

Global seed set to 0
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [5]:
from pytorch_lightning.utilities.seed import seed_everything
for i in range(10):
    seed_everything(i+1)
    model = SimVI(adata,kl_weight=1,kl_gatweight=0.01,lam_mi=1000,permutation_rate=0.5,n_spatial=20,n_intrinsic=20)
    train_loss, val_loss = model.train(edge_index,max_epochs=100,batch_size=1000,use_gpu=True,mae_epochs=25,device='cuda:2')
    adata.obsm['simvi_z'+str(i)] = model.get_latent_representation(edge_index,representation_kind='intrinsic',give_mean=True)
    adata.obsm['simvi_s'+str(i)] = model.get_latent_representation(edge_index,representation_kind='interaction',give_mean=True)
    adata.obsm['simvi_'+str(i)] = np.hstack((adata.obsm['simvi_z'+str(i)],adata.obsm['simvi_s'+str(i)]))
    np.save('Benchmarking_results/slidetag/'+'simvi_z'+str(i)+'.npy',adata.obsm['simvi_z'+str(i)])
    np.save('Benchmarking_results/slidetag/'+'simvi_s'+str(i)+'.npy',adata.obsm['simvi_s'+str(i)])
    np.save('Benchmarking_results/slidetag/'+'simvi_'+str(i)+'.npy',adata.obsm['simvi_'+str(i)])

Global seed set to 1
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:44<00:00,  2.24it/s, train_loss=1.09e+3, val_loss=1119.0975]
Global seed set to 2
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:46<00:00,  2.14it/s, train_loss=1.09e+3, val_loss=1121.5317]
Global seed set to 3
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:43<00:00,  2.30it/s, train_loss=1.08e+3, val_loss=1116.9712]
Global seed set to 4
Epoch 100/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

In [6]:
from simvi.model import SimVIGraphOnly
SimVIGraphOnly.setup_anndata(adata,batch_key='cell_type')
edge_index = SimVIGraphOnly.extract_edge_index(adata,n_neighbors=10)

## likelihood

from pytorch_lightning.utilities.seed import seed_everything
for i in range(10):
    seed_everything(i+1)
    model = SimVIGraphOnly(adata,kl_weight=1,kl_gatweight=1,lam_mi=1000,permutation_rate=0,n_spatial=20)
    train_loss, val_loss = model.train(edge_index,max_epochs=100,batch_size=1000,use_gpu=True)
    adata.obsm['simvigoct_s'+str(i)] = model.get_latent_representation(edge_index,representation_kind='interaction',give_mean=True)
    np.save('Benchmarking_results/slidetag/'+'simvigoct_s'+str(i)+'.npy',adata.obsm['simvigoct_s'+str(i)])

Global seed set to 1
Epoch 100/100: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:10<00:00,  9.69it/s, train_loss=1.09e+3, val_loss=1133.706]
Global seed set to 2
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:10<00:00,  9.71it/s, train_loss=1.09e+3, val_loss=1131.5444]
Global seed set to 3
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:09<00:00, 10.02it/s, train_loss=1.08e+3, val_loss=1130.9393]
Global seed set to 4
Epoch 100/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

In [7]:
from simvi.model import SimVIGraphOnly
SimVIGraphOnly.setup_anndata(adata)
edge_index = SimVIGraphOnly.extract_edge_index(adata,n_neighbors=10)

## likelihood

from pytorch_lightning.utilities.seed import seed_everything
for i in range(10):
    seed_everything(i+1)
    model = SimVIGraphOnly(adata,kl_weight=1,kl_gatweight=1,lam_mi=1000,permutation_rate=0,n_spatial=20)
    train_loss, val_loss = model.train(edge_index,max_epochs=100,batch_size=1000,use_gpu=True)
    adata.obsm['simvigo_s'+str(i)] = model.get_latent_representation(edge_index,representation_kind='interaction',give_mean=True)
    np.save('Benchmarking_results/slidetag/'+'simvigo_s'+str(i)+'.npy',adata.obsm['simvigo_s'+str(i)])

Global seed set to 1
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:10<00:00,  9.94it/s, train_loss=1.12e+3, val_loss=1208.0618]
Global seed set to 2
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:10<00:00,  9.87it/s, train_loss=1.12e+3, val_loss=1221.0164]
Global seed set to 3
Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:10<00:00,  9.66it/s, train_loss=1.13e+3, val_loss=1199.0426]
Global seed set to 4
Epoch 100/100: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

In [8]:
import scvi
scvi.model.SCVI.setup_anndata(adata)
for i in range(10):
    seed_everything(i)
    vae = scvi.model.SCVI(adata, n_layers=2, n_latent=20)
    vae.train(max_epochs=100,use_gpu=True,batch_size=1000)
    adata.obsm['scvi'+str(i)] = vae.get_latent_representation(adata)
    np.save('Benchmarking_results/slidetag/'+'scvi'+str(i)+'.npy',adata.obsm['scvi'+str(i)])

Global seed set to 0
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/data_loading.py:432: UserWarning: The number of training samples (6) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:11<00:00,  8.74it/s, loss=1.08e+03, v_num=1]


Global seed set to 1
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/data_loading.py:432: UserWarning: The number of training samples (6) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.44it/s, loss=1.08e+03, v_num=1]


Global seed set to 2
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/data_loading.py:432: UserWarning: The number of training samples (6) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


...

Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:12<00:00,  7.95it/s, loss=1.09e+03, v_num=1]


Global seed set to 8
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/data_loading.py:432: UserWarning: The number of training samples (6) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.54it/s, loss=1.08e+03, v_num=1]


Global seed set to 9
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/data_loading.py:432: UserWarning: The number of training samples (6) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 100/100: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.56it/s, loss=1.08e+03, v_num=1]
